<br>
<img align="left" width="300" src="rflogo.png">

# Selecting Segments for Vacancies in Shopping Malls 


<img align="left" width="80" height="200" src="https://img.shields.io/badge/python-v3.6-blue.svg">
<br>

### Notebook by [Marco Tavora](https://marcotavora.me/)


## Table of contents

1. [Introduction](#Introduction)
1. [Generating Data](#Generating-Data) 
2. [Detailed Steps](#Detailed-Steps)

## Introduction
[[go back to the top]](#Table-of-contents)

The problem can be stated as follows:

> *Consider a shopping center with several vacant spots. We want to determine which types of store (segments) should occupy each of the spots given a set of constraints.*

The constraints are:
- the fraction of the corridor area allocated to each store must have an upper limit (dependent on the store segment)
- the flux of the store must fall within some proper interval (one for each corridor).

In this notebook I will develop some solutions to this problem.

<a href="https://en.wikipedia.org/wiki/Shopping_mall"><img src="planta-comercial-1.png"></a>
<center>Sample mall used in this analysis.</center>                        

## Data Handling
[[go back to the top]](#Table-of-contents)

The stores and vacant spots are give by:

In [49]:
%run modules_algorithm.ipynb

In [50]:
lojas_vagas = data_builder()[1].iloc[:,:-1]
lojas_ocupadas = data_builder()[0]

In [51]:
# the line below is to remove unwanted spaces
lojas_ocupadas['segmento'] = [el.strip() for el 
                              in lojas_ocupadas['segmento']]

The vacancies are:

In [52]:
lojas_vagas.head()
lojas_vagas.tail()
print('There are {} vacant spots'.format(lojas_vagas.shape[0]))

,corredor,size_classes,area_m2
loja,,,
2,2,1,44.0
7,0,4,252.0
15,0,2,65.0
19,1,1,55.0
25,1,1,51.0


,corredor,size_classes,area_m2
loja,,,
111,0,1,39.0
114,0,2,84.0
ANC1,1,5,1166.0
SA1,1,5,528.0
SA4,1,5,676.0


There are 25 vacant spots


The stores already in operation are:

In [53]:
lojas_ocupadas.head()
print('There are {} stores'.format(lojas_ocupadas.shape[0]))

,segmento_code,corredor,size_classes,area_m2,segmento
loja,,,,,
1,0,0,4,294.0,Ótica
3,1,0,1,40.0,Serviço
4,0,0,1,40.0,Ótica
5,2,0,1,38.0,Eletrônicos
6,2,0,1,38.0,Eletrônicos


There are 90 stores


The number of segments, corridors and size classes are below.

In [54]:
lojas_ocupadas['segmento'] = [el.strip() for el in lojas_ocupadas['segmento']]
corredores_lojas_ocupadas = lojas_ocupadas['corredor'].unique().tolist()
segmentos = lojas_ocupadas['segmento'].unique().tolist()
size_classes = lojas_ocupadas['size_classes'].unique().tolist()

(s1, s2) = (list(lojas_ocupadas['segmento']), 
            list(lojas_ocupadas['segmento_code']))

dicionario_segmentos = dict_builder(s1, s2)

codes = pd.DataFrame(list(dicionario_segmentos.items()), 
             columns = ['segmento', 'segmento_code'])
codes.index = codes['segmento']
codes = codes.drop('segmento', axis=1)
codes.T

(I, J, K) = (len(segmentos), 
             len(corredores_lojas_ocupadas), 
             len(size_classes))

print(' numero de segmentos, numero de corredores, numeros de tipos de tamanho:',I,',',J,',',K) 

segmento,Ótica,Serviço,Eletrônicos,Vestuário,Colchões,Acessórios Femininos,Perfumaria,Pet Shop,Cafeteria,Sobremesa,Brinquedos,Calçados,Papelaria,Drogaria,Presentes,Cama e Banho,Alimentação,Departamento,Livraria,Art. Esportivos,Supermercado
segmento_code,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20


 numero de segmentos, numero de corredores, numeros de tipos de tamanho: 21 , 3 , 5


## Vacancies
[[go back to the top]](#Table-of-contents)

For simplicity I will start considering that there is only one vacancy in the mall and will drop the remaining rows from `lojas_vagas`. I will keep specifically vacancy 2. The column `jk` is the concatenation of columns `corredor` and `size_classes`.

In [55]:
lojas_vagas['loja'] = lojas_vagas.index
lojas_vagas = lojas_vagas[(lojas_vagas['loja'] == 19)|(lojas_vagas['loja'] == 114)]
lojas_vagas = lojas_vagas.iloc[:,:-1]
lojas_vagas['jk'] = list(zip(lojas_vagas['corredor'], lojas_vagas['size_classes']))
lojas_vagas

,corredor,size_classes,area_m2,jk
loja,,,,
19,1,1,55.0,"(1, 1)"
114,0,2,84.0,"(0, 2)"


The total area occupied by vacancies is:

In [56]:
A_v = {}
J = 2
for j in range(J):
    A_v[j] = lojas_vagas[lojas_vagas['corredor'] == j]['area_m2'].sum()
    print('Corridor {} total vacant area:'.format(j), A_v[j])
    
df_A_v = pd.DataFrame(list(A_v.items()), columns = ['jk', 'area'])
df_A_v

Corridor 0 total vacant area: 84.0
Corridor 1 total vacant area: 55.0


,jk,area
0,0,84.0
1,1,55.0


### Area matrices for occupied stores

In [57]:
A_oc = {}

for j in range(J):
    A_oc[j] = lojas_ocupadas[lojas_ocupadas['corredor'] == j]['area_m2'].sum()
    print('Corridor {} occupied area:'.format(j), A_oc[j])

Corridor 0 occupied area: 9161.0
Corridor 1 occupied area: 6752.0


Adding column concatenating `corredor` and `segmento_code`:

In [58]:
lojas_ocupadas['ij'] = list(zip(lojas_ocupadas['segmento_code'], lojas_ocupadas['corredor']))

In [59]:
cc = pd.read_csv('cc_classes.csv')
cc['segmento'] = [el.strip() for el in cc['segmento']]
cc['segmento_code'] = cc['segmento'].map(dicionario_segmentos)
cc.head()

,segmento,cc,segmento_code
0,Acessórios Femininos,cca,5
1,Alimentação,cca,16
2,Art. Esportivos,cca,19
3,Brinquedos,ccm,10
4,Cafeteria,ccb,8


In [60]:
s1, s2 = list(cc['cc']), list(cc['segmento_code'])
dicionario_cc = dict_builder(s2, s1)
lojas_ocupadas['cc'] = lojas_ocupadas['segmento_code'].map(dicionario_cc)
lojas_ocupadas.head()

,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc
loja,,,,,,,
1,0,0,4,294.0,Ótica,"(0, 0)",cca
3,1,0,1,40.0,Serviço,"(1, 0)",ccm
4,0,0,1,40.0,Ótica,"(0, 0)",cca
5,2,0,1,38.0,Eletrônicos,"(2, 0)",cca
6,2,0,1,38.0,Eletrônicos,"(2, 0)",cca


In [61]:
lojas_ocupadas.isnull().any().unique()

array([False])

### Corridor total areas

In [62]:
G = {}

for c in lojas_ocupadas['corredor'].tolist():
    area_ocupada = lojas_ocupadas[lojas_ocupadas['corredor'] == c]['area_m2'].sum()
    area_vaga = lojas_vagas[lojas_vagas['corredor'] == c]['area_m2'].sum()
    area = area_ocupada + area_vaga
    if area > 0:
        G[c] = area
    else:
        pass

print(' total areas by location:\n\n', G)

 total areas by location:

 {0: 9245.0, 1: 6807.0, 2: 1226.0}


###  Maximum percentage $f_{ij}$ of stores of segment $i$ in corridor $j$ classified by `cc`

In [63]:
s = list(lojas_ocupadas['segmento'])
dict_segmentos_cc = dict_builder(s, list(lojas_ocupadas['cc']))
dict_cc_pct = {'cca': 10000000.0, 'ccm': 0.35, 'ccb':0.1}

phi_values = [0.37, 0.45, 0.18]

dict_corredor_phi = dict_builder(list(range(4)), phi_values)
lojas_ocupadas['phi'] = lojas_ocupadas['corredor'].map(dict_corredor_phi)

dict_cc_pct = {'cca': 10000000.0, 'ccm': 0.35, 'ccb':0.1}

dict_phi = {'ccb':(0.0, 0.21) ,'ccm':(0.22, 0.33), 'cca':(0.34, 1.0)}

lojas_ocupadas['flux_range'] = lojas_ocupadas['cc'].map(dict_phi)


lst = [(lojas_ocupadas['phi'][i] > lojas_ocupadas['flux_range'][i][0]
        and lojas_ocupadas['phi'][i] < lojas_ocupadas['flux_range'][i][1])
        for i in range(lojas_ocupadas.shape[0])]

lojas_ocupadas['T/F'] = lst
lojas_ocupadas['Substituir?'] = ['Sim' if v == False else 'Nao' for v in list(lojas_ocupadas['T/F'])]

lojas_ocupadas.head()



f = {}

for el in list(lojas_ocupadas['segmento_code'].unique()):
    df = lojas_ocupadas[lojas_ocupadas['segmento_code']== el]
    for corredor in list(lojas_ocupadas['corredor'].unique()):
        aux = df[df['corredor'] == corredor]
        cc =  dicionario_cc[el]
        f[(el, corredor)] = dict_cc_pct[cc]


df_f = pd.DataFrame(list(f.items()), columns = ['ij', 'f'])

df_f.head()

,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc,phi,flux_range,T/F,Substituir?
loja,,,,,,,,,,,
1,0,0,4,294.0,Ótica,"(0, 0)",cca,0.37,"(0.34, 1.0)",True,Nao
3,1,0,1,40.0,Serviço,"(1, 0)",ccm,0.37,"(0.22, 0.33)",True,Nao
4,0,0,1,40.0,Ótica,"(0, 0)",cca,0.37,"(0.34, 1.0)",True,Nao
5,2,0,1,38.0,Eletrônicos,"(2, 0)",cca,0.37,"(0.34, 1.0)",False,Sim
6,2,0,1,38.0,Eletrônicos,"(2, 0)",cca,0.37,"(0.34, 1.0)",True,Nao


,ij,f
0,"(0, 0)",10000000.00
1,"(0, 1)",10000000.00
2,"(0, 2)",10000000.00
3,"(1, 0)",0.35
4,"(1, 1)",0.35


In [64]:
lojas_ocupadas.isnull().any().unique()

array([False])

Adding a corridor flux column to `lojas_vagas`:

In [65]:
lojas_vagas['phi'] = lojas_vagas['corredor'].map(dict_corredor_phi)

In [66]:
lojas_vagas

,corredor,size_classes,area_m2,jk,phi
loja,,,,,
19,1,1,55.0,"(1, 1)",0.45
114,0,2,84.0,"(0, 2)",0.37


### Summary

In [67]:
print('numero de segmentos, numero de corredores, numeros de tipos de tamanho:',I,',',J,',',K)
print('segmentos:',segmentos,'|','size_classes:\n',size_classes,'|','corredores com lojas ocupadas:',
      corredores_lojas_ocupadas,'\n')

num_vacancies = lojas_vagas.shape[0]

print('num_vacancies:', num_vacancies,'\n')


print('A_v{j,k} => areas of vacancies in corridor j and size class k:\n')

df_A_v.head(2)

print('Total areas by corridor = (corridor, size class):', G,'\n')

print('f_{i,j} maximum % of stores of segmento i in corridor j:\n')

df_f.head(2)

print('Occupied area:\n')
print(A_oc,'\n')
print('Vacant area:\n')
print(A_v,'\n')

print("Vacancies profile:",'\n')
lojas_vagas.head(2)
print("Stores profile:",'\n')
print('Vacant stores:', list(lojas_vagas.index))
print('Number of vacant stores:', len(list(lojas_vagas.index)))
lojas_ocupadas.head(2)
print('Occupied stores:',list(lojas_ocupadas.index))
print('Number of open stores:', len(list(lojas_ocupadas.index)))

numero de segmentos, numero de corredores, numeros de tipos de tamanho: 21 , 2 , 5
segmentos: ['Ótica', 'Serviço', 'Eletrônicos', 'Vestuário', 'Colchões', 'Acessórios Femininos', 'Perfumaria', 'Pet Shop', 'Cafeteria', 'Sobremesa', 'Brinquedos', 'Calçados', 'Papelaria', 'Drogaria', 'Presentes', 'Cama e Banho', 'Alimentação', 'Departamento', 'Livraria', 'Art. Esportivos', 'Supermercado'] | size_classes:
 [4, 1, 3, 2, 5] | corredores com lojas ocupadas: [0, 1, 2] 

num_vacancies: 2 

A_v{j,k} => areas of vacancies in corridor j and size class k:



,jk,area
0,0,84.0
1,1,55.0


Total areas by corridor = (corridor, size class): {0: 9245.0, 1: 6807.0, 2: 1226.0} 

f_{i,j} maximum % of stores of segmento i in corridor j:



,ij,f
0,"(0, 0)",10000000.0
1,"(0, 1)",10000000.0


Occupied area:

{0: 9161.0, 1: 6752.0} 

Vacant area:

{0: 84.0, 1: 55.0} 

Vacancies profile: 



,corredor,size_classes,area_m2,jk,phi
loja,,,,,
19,1,1,55.0,"(1, 1)",0.45
114,0,2,84.0,"(0, 2)",0.37


Stores profile: 

Vacant stores: [19, 114]
Number of vacant stores: 2


,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc,phi,flux_range,T/F,Substituir?
loja,,,,,,,,,,,
1,0,0,4,294.0,Ótica,"(0, 0)",cca,0.37,"(0.34, 1.0)",True,Nao
3,1,0,1,40.0,Serviço,"(1, 0)",ccm,0.37,"(0.22, 0.33)",True,Nao


Occupied stores: [1, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 53, 54, 55, 57, 58, 60, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 77, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 100, 103, 104, 105, 106, 107, 108, 109, 113, 'ANC2', 'ANC3', 'ANC4', 'SA2', 'SA3', 'SA5', 'SUPER']
Number of open stores: 90


In [68]:
for j in range(J):
    print('Corridor {} occupied area:'.format(j), A_oc[j],'\n')
    
print('Total corridor {} are G:', G[j])

Corridor 0 occupied area: 9161.0 

Corridor 1 occupied area: 6752.0 

Total corridor {} are G: 6807.0


In [69]:
RHS = [G[j] - A_oc[j] for j in range(2)]

for j in range(2):
    print('RHS of the constraint {}'.format(j),'is', RHS[j])

RHS of the constraint 0 is 84.0
RHS of the constraint 1 is 55.0


In [70]:
lojas_vagas.head()
lojas_ocupadas.head()

j, k = list(lojas_vagas['corredor'])[0], list(lojas_vagas['size_classes'])[0]
print('')
print('vaga esta no corredor {} and size class {}'.format(j,k),'\n')
print('Occupied area:', A_oc)
a11 = lojas_vagas[(lojas_vagas['corredor'] == j) & (lojas_vagas['size_classes'] == k)]['area_m2'].sum()
print('a_11:', a11,'\n')
print('f_{i,j} maximum % of stores of segmento i in corridor j:\n')

f_new = {}

for i in range(df_f.shape[0]):
    if df_f.iloc[i,0][0] == 2:
        f_new[df_f.iloc[i,0]] = df_f.iloc[i,1]
print(f_new)    

,corredor,size_classes,area_m2,jk,phi
loja,,,,,
19,1,1,55.0,"(1, 1)",0.45
114,0,2,84.0,"(0, 2)",0.37


,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc,phi,flux_range,T/F,Substituir?
loja,,,,,,,,,,,
1,0,0,4,294.0,Ótica,"(0, 0)",cca,0.37,"(0.34, 1.0)",True,Nao
3,1,0,1,40.0,Serviço,"(1, 0)",ccm,0.37,"(0.22, 0.33)",True,Nao
4,0,0,1,40.0,Ótica,"(0, 0)",cca,0.37,"(0.34, 1.0)",True,Nao
5,2,0,1,38.0,Eletrônicos,"(2, 0)",cca,0.37,"(0.34, 1.0)",False,Sim
6,2,0,1,38.0,Eletrônicos,"(2, 0)",cca,0.37,"(0.34, 1.0)",True,Nao



vaga esta no corredor 1 and size class 1 

Occupied area: {0: 9161.0, 1: 6752.0}
a_11: 55.0 

f_{i,j} maximum % of stores of segmento i in corridor j:

{(2, 0): 10000000.0, (2, 1): 10000000.0, (2, 2): 10000000.0}


In [71]:
lojas_ocupadas[(lojas_ocupadas['corredor'] == j) & (lojas_ocupadas['segmento_code'] == 1)]
lojas_ocupadas[lojas_ocupadas['corredor'] == j]

,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc,phi,flux_range,T/F,Substituir?
loja,,,,,,,,,,,
44,1,1,1,53.0,Serviço,"(1, 1)",ccm,0.45,"(0.22, 0.33)",True,Nao
50,1,1,2,94.0,Serviço,"(1, 1)",ccm,0.45,"(0.22, 0.33)",False,Sim
57,1,1,1,42.0,Serviço,"(1, 1)",ccm,0.45,"(0.22, 0.33)",False,Sim
58,1,1,3,131.0,Serviço,"(1, 1)",ccm,0.45,"(0.22, 0.33)",True,Nao
109,1,1,4,262.0,Serviço,"(1, 1)",ccm,0.45,"(0.22, 0.33)",False,Sim


,segmento_code,corredor,size_classes,area_m2,segmento,ij,cc,phi,flux_range,T/F,Substituir?
loja,,,,,,,,,,,
20,2,1,1,42.0,Eletrônicos,"(2, 1)",cca,0.45,"(0.34, 1.0)",True,Nao
21,7,1,2,75.0,Pet Shop,"(7, 1)",ccb,0.45,"(0.0, 0.21)",False,Sim
22,8,1,2,76.0,Cafeteria,"(8, 1)",ccb,0.45,"(0.0, 0.21)",True,Nao
23,2,1,2,70.0,Eletrônicos,"(2, 1)",cca,0.45,"(0.34, 1.0)",True,Nao
24,9,1,2,70.0,Sobremesa,"(9, 1)",ccb,0.45,"(0.0, 0.21)",True,Nao
26,4,1,1,58.0,Colchões,"(4, 1)",cca,0.45,"(0.34, 1.0)",True,Nao
27,5,1,2,90.0,Acessórios Femininos,"(5, 1)",cca,0.45,"(0.34, 1.0)",True,Nao
29,9,1,2,100.0,Sobremesa,"(9, 1)",ccb,0.45,"(0.0, 0.21)",False,Sim
31,2,1,3,206.0,Eletrônicos,"(2, 1)",cca,0.45,"(0.34, 1.0)",True,Nao


In [73]:
j,k

(1, 1)

In [72]:
Aj = {}

for i in set(list(lojas_ocupadas['segmento_code'])):
    aux = lojas_ocupadas[(lojas_ocupadas['corredor'] == j) & (aux['segmento_code'] == i)]
    aux = aux[aux['segmento_code'] == i]
    aux_area = aux['area_m2']       
    Aj[i] = aux_area.sum()
    print('A_(ij) for i = {} = '.format(i), Aj[i])

A_(ij) for i = 0 =  0.0
A_(ij) for i = 1 =  0.0
A_(ij) for i = 2 =  0.0
A_(ij) for i = 3 =  0.0
A_(ij) for i = 4 =  0.0
A_(ij) for i = 5 =  0.0
A_(ij) for i = 6 =  0.0
A_(ij) for i = 7 =  0.0
A_(ij) for i = 8 =  0.0
A_(ij) for i = 9 =  0.0
A_(ij) for i = 10 =  0.0
A_(ij) for i = 11 =  0.0
A_(ij) for i = 12 =  0.0
A_(ij) for i = 13 =  0.0
A_(ij) for i = 14 =  0.0
A_(ij) for i = 15 =  0.0
A_(ij) for i = 16 =  0.0
A_(ij) for i = 17 =  0.0
A_(ij) for i = 18 =  0.0
A_(ij) for i = 19 =  0.0
A_(ij) for i = 20 =  0.0


## Building the optimization problem

We will determine which stores can be allocated in this vacancy based on a set of constraints. 

The first is:

$$A_{ij}^{{\text{(oc)}}} + \sum\limits_{k = 1}^K {a_{jk}^{(s)}} {x_{ijk}} \le {f_{ij}}{G_j}\,$$

where $i$ is the segment,  $j$ is the corridor number and $k$ the size class number. Here, $x_{ijk}$ is the number of stores with indexes $(i,j,k)$, which can be 0 or 1, $G_{j}^{(s)}$ is the total area of corridor $j$, $a_{ij}^{\text{oc}}$ is the area of segment $i$ stores in corridor $j$. In our case this becomes only one condition:

$$A_{i{j_{\text{new}}}}^{{\text{(oc)}}} + {a_{11}^{(s)}} {x_{i{j_{\text{new}}}{k_{\text{new}}}}} \le {f_{i{j_{\text{new}}}}}{G_{j_{\text{new}}}}\,$$


The condition on the flux restricts the segments to:

In [39]:
lojas_ocupadas_c1 = lojas_ocupadas[lojas_ocupadas['corredor'] == j]

In [40]:
possible_segments = list(lojas_ocupadas_c1[(lojas_ocupadas_c1['phi'] == phi_values[j]) &
                                           (lojas_ocupadas_c1['flux_range'] == dict_phi['ccb'])]['segmento_code'].unique())

In [41]:
possible_segments

[8, 20]

In [42]:
print('possible segments:', possible_segments)

possible segments: [8, 20]


In [43]:
for k in possible_segments:
    print('cc of segment {}:'.format(k), dicionario_cc[k])

cc of segment 8: ccb
cc of segment 20: ccb


$$A_{i1}^{{\text{(oc)}}} + {a_{11}^{(s)}} {x_{i11}} \le {f_{i1}}{G_1}\,$$

In [44]:
for i in possible_segments:
    print(A1[i] + a11, G[1], f[i,1])

84.0 6752.0 0.1
84.0 6752.0 0.1


In [46]:
def retail_matching(a11, possible_segments):

    stores = [(i, j, k) for i in range(I)]
    
    x = pulp.LpVariable.dicts('x', stores, cat = 'Binary')

    prob = pulp.LpProblem('Matching', LpMaximize)

    for i in possible_segments:
        if f[i,1] < 1:
            prob += A1[i] + a11*x[i,j,k] <= f[i,j]*G[j]

    return prob

prob = retail_matching(a11, possible_segments)

prob

Matching:
MAXIMIZE
None
SUBJECT TO
_C1: 84 x_(8,_0,_20) <= 924.5

_C2: 84 x_(20,_0,_20) <= 924.5

VARIABLES
0 <= x_(20,_0,_20) <= 1 Integer
0 <= x_(8,_0,_20) <= 1 Integer